In [ ]:
#!/usr/bin/env python3
import serial
import onnx

import numpy as np
import pandas as pd
import tensorflow as tf
import onnxruntime

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from onnxruntime import InferenceSession
from onnx import numpy_helper


In [ ]:
model_path = 'rpmodel.onnx'
onnx_model = onnx.load(model_path)
try:
	onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
	pass
else:
	print('model is valid')

def predict(line):
    session = onnxruntime.InferenceSession('rpmodel.onnx', None)
    line = line - 200
    line = line/833
    test_data = np.float32(line)
    test_data = np.reshape(test_data, (-1, 140))
    print(test_data)
    ortvalue = onnxruntime.OrtValue.ortvalue_from_numpy(test_data)
 #   ortvalue.device_name()  # 'cpu'
 #   ortvalue.shape()        # shape of the numpy array X
 #   ortvalue.data_type()    # 'tensor(float)'
 #   ortvalue.is_tensor()    # 'True'
 #   np.array_equal(ortvalue.numpy(), test_data)  # 'True'

    Input_name = session.get_inputs()[0].name

    output_name = session.get_outputs()[0].name
    
    result = session.run([], {Input_name: ortvalue})

    normal = lambda x: x[1] > 0.3

    #print("generated input numpy data type", test_data.dtype)
    #tensortest = numpy_helper.from_array(test_data)
    #print(test_data)
    #print(""tensortest.data_type)
    # Input_type = session.get_inputs()[0].type
    # print("Input type", Input_type)
    #Input_name = session.get_inputs()[0].name
    #print(test_data.shape)
    result = session.run([output_name], {Input_name: test_data})
    pred = result[0]
    print(pred)
    #prediction=np.argmax(np.array(result).squeeze(), axis=0)
    prediction = [normal(i) for i in pred]
    if prediction[0]==True:
        print('normal')
        time.sleep(2)
    else :
        print('abnormal')
 #   for i in prediction:
 #     print('Normal' if i else 'Abnormal')
    #print('normal' if prediction == True else 'abnormal')
    #print(pred)



#main

if __name__ == '__main__':
    ser = serial.Serial('/dev/ttyACM0', 115200, timeout=1)
    ser.reset_input_buffer()
    buffer_input = np.zeros(140)
    
    
    time_start = time.time()
    
    count = 0
    while True:
        if ser.in_waiting > 0:
        
            temp = ser.readline().decode('utf-8').rstrip()
            
            buffer_input[count] = temp
            count += 1 
            
            if count >= 139:
                count = 0
                if buffer_input.max()>500:
                    time_start = time.time()
                if time.time()-time_start > 5.:
                    print('no heart beat detected')
                    continue
                
                
             #   print(len(line))
                print(buffer_input)
                predict(buffer_input) 
            


